In [ ]:
!gdown --id 1dWfWzaISPR1Cv4tR0EtNhD1jQODV0Kp9 # train test key
!gdown --id 1ji2idP3o37gQmyR-x-n849LbhmxilxPG # full dataset
!gdown --id 1bIiROVpdUmX9af-La19asq3UXoNlR3-S # util

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dWfWzaISPR1Cv4tR0EtNhD1jQODV0Kp9
To: /content/train_test_key.csv
100% 7.86k/7.86k [00:00<00:00, 5.03MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ji2idP3o37gQmyR-x-n849LbhmxilxPG
To: /content/USDM_SMAP_SET_NOAA_STL_full.csv
100% 55.0M/55.0M [00:00<00:00, 139MB/s]


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from util import *
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
import tensorflow as tf
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor


import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

import sklearn.linear_model as lm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import sklearn.impute as im
from sklearn.preprocessing import PolynomialFeatures

from datetime import timedelta
from datetime import datetime
from tqdm import tqdm

# Data processing

## train/test keys

In [ ]:
ttkey=pd.read_csv('train_test_key.csv',dtype=str)
trkey=ttkey.iloc[:,0]
trkey=[int(i) for i in list(trkey.to_numpy())]
tekey=ttkey.iloc[:,2].dropna(axis=0)
tekey=[int(i) for i in list(tekey.to_numpy())]

In [ ]:
len(trkey),len(tekey)

(360, 90)

## subsetting dataset

In [ ]:
df=pd.read_csv('USDM_SMAP_SET_NOAA_STL_full.csv')
df[df==-9999]=np.nan
df['MapDate']=pd.to_datetime(df['MapDate'])

# Threshold
df['Y']=np.where(df['None']==100,0,1)

cache=df[['MapDate', 'FIPS STR', 'Y', 'roff', 'evap', 'Region']]
cache=cache.rename(columns={'FIPS STR':'FIPS_STR'})
dfs=cache.dropna()

dfs['dT']=[np.timedelta64(dfs['MapDate'].to_numpy()[i]-dfs['MapDate'].to_numpy()[0],'W').astype(int) for i in range(dfs.shape[0])]
dfs

,MapDate,FIPS_STR,Y,roff,evap,Region,dT
157,2014-12-30,1007,0,0.001730,-3.701072e-04,Southeast,0
158,2015-01-06,1007,0,0.000068,-9.940644e-05,Southeast,1
159,2015-01-13,1007,0,0.000077,-1.348449e-04,Southeast,2
160,2015-01-20,1007,0,0.000395,-3.874424e-04,Southeast,3
161,2015-01-27,1007,0,0.000193,-2.163508e-04,Southeast,4
...,...,...,...,...,...,...,...
238937,2021-11-30,56043,1,0.000002,-2.076237e-05,NorthernRR,361
238938,2021-12-07,56043,1,0.000002,-8.488739e-06,NorthernRR,362
238939,2021-12-14,56043,1,0.000002,-5.072568e-05,NorthernRR,363
238940,2021-12-21,56043,1,0.000002,-2.681730e-05,NorthernRR,364


In [ ]:
yy=dfs.Y.to_numpy()
np.sum(yy)/len(yy)

0.4983630898049095

In [ ]:
# check if date matched
# dfs.groupby('FIPS_STR').count(),dfs.groupby('FIPS_STR').count().nunique()

## WARNING: county 45019 is NaN. please check original datasets.

In [ ]:
dfs_fips=dfs['FIPS_STR'].unique()
for i in trkey:
  if i in dfs_fips: pass
  else: print(i)

45019


In [ ]:
df[df['FIPS STR']==45019].roff

185850   NaN
185851   NaN
185852   NaN
185853   NaN
185854   NaN
          ..
186376   NaN
186377   NaN
186378   NaN
186379   NaN
186380   NaN
Name: roff, Length: 531, dtype: float64

#### TRAIN ON TRAINING SET'S WEEK 0   TO 285
#### TEST  ON TESTING SET'S  WEEK 286 TO 365

In [ ]:
print('train')
print(timekey(horizon=4, nhist=8,thin=False,total_days=286,test=False)[:5])
print('test:')
print(timekey(horizon=4, nhist=8,thin=False,total_days=286,test=True)[:5])

train
[(array([0, 1, 2, 3, 4, 5, 6, 7]), array([ 8,  9, 10, 11])), (array([1, 2, 3, 4, 5, 6, 7, 8]), array([ 9, 10, 11, 12])), (array([2, 3, 4, 5, 6, 7, 8, 9]), array([10, 11, 12, 13])), (array([ 3,  4,  5,  6,  7,  8,  9, 10]), array([11, 12, 13, 14])), (array([ 4,  5,  6,  7,  8,  9, 10, 11]), array([12, 13, 14, 15]))]
test:
[(array([286, 287, 288, 289, 290, 291, 292, 293]), array([294, 295, 296, 297])), (array([287, 288, 289, 290, 291, 292, 293, 294]), array([295, 296, 297, 298])), (array([288, 289, 290, 291, 292, 293, 294, 295]), array([296, 297, 298, 299])), (array([289, 290, 291, 292, 293, 294, 295, 296]), array([297, 298, 299, 300])), (array([290, 291, 292, 293, 294, 295, 296, 297]), array([298, 299, 300, 301]))]


# $X_{t-17~t-1}: Y_{t~t+6}$

In [ ]:
horizon_=6
nhist_=18
feature_name=['Y', 'roff', 'evap']

x_tr,y_tr,x_te,y_te=trte_split(horizon=horizon_,nhist=nhist_,
                               thin=False,
                               trkey=trkey,tekey=tekey,
                               dfs=dfs,
                               feature_name=feature_name)

model=dnnmodel(dim=54,out_dim=6)

model.fit(x_tr,y_tr,batch_size=128,epochs=4)

print('test accuracy:', model.evaluate(x_te,y_te)[1]*100)

 40%|████      | 145/360 [02:52<06:48,  1.90s/it]

passed county: 45019


100%|██████████| 90/90 [00:10<00:00,  8.73it/s]


Epoch 1/4
735/735 [==============================] - 5s 6ms/step - loss: 0.3972 - binary_accuracy: 0.8493
Epoch 2/4
735/735 [==============================] - 4s 6ms/step - loss: 0.3799 - binary_accuracy: 0.8620
Epoch 3/4
735/735 [==============================] - 4s 6ms/step - loss: 0.3775 - binary_accuracy: 0.8620
Epoch 4/4
158/158 [==============================] - 0s 2ms/step - loss: 0.3364 - binary_accuracy: 0.9003
test accuracy: 90.02976417541504


# $X_{t-17~t-1}: Y_{t+6}$

In [ ]:
horizon_=6
nhist_=18
feature_name=['Y', 'roff', 'evap']

x_tr,y_tr,x_te,y_te=trte_split(horizon=horizon_,nhist=nhist_,
                               thin=True,
                               trkey=trkey,tekey=tekey,
                               dfs=dfs,
                               feature_name=feature_name)

model=dnnmodel(dim=54,out_dim=1)

model.fit(x_tr,y_tr,batch_size=128,epochs=4)

print('test accuracy:', model.evaluate(x_te,y_te)[1]*100)

 40%|████      | 145/360 [02:28<05:33,  1.55s/it]

passed county: 45019


100%|██████████| 90/90 [00:10<00:00,  8.65it/s]


Epoch 1/4
735/735 [==============================] - 6s 7ms/step - loss: 0.5153 - binary_accuracy: 0.7796
Epoch 2/4
735/735 [==============================] - 5s 7ms/step - loss: 0.5069 - binary_accuracy: 0.7862
Epoch 3/4
735/735 [==============================] - 5s 7ms/step - loss: 0.5055 - binary_accuracy: 0.7863
Epoch 4/4
158/158 [==============================] - 0s 2ms/step - loss: 0.4415 - binary_accuracy: 0.8514
test accuracy: 85.13888716697693


# $X_{t-7~t-1}: Y_{t+1~t+4}$

In [ ]:
horizon_=4
nhist_=8
feature_name=['Y', 'roff', 'evap']

x_tr,y_tr,x_te,y_te=trte_split(horizon=horizon_,nhist=nhist_,
                               thin=False,
                               trkey=trkey,tekey=tekey,
                               dfs=dfs,
                               feature_name=feature_name)

model=dnnmodel(dim=24,out_dim=4)

model.fit(x_tr,y_tr,batch_size=128,epochs=4)

print('test accuracy:', model.evaluate(x_te,y_te)[1]*100)

 40%|████      | 145/360 [01:53<03:53,  1.09s/it]

passed county: 45019


100%|██████████| 90/90 [00:11<00:00,  7.96it/s]


Epoch 1/4
769/769 [==============================] - 5s 6ms/step - loss: 0.3413 - binary_accuracy: 0.8856
Epoch 2/4
769/769 [==============================] - 5s 6ms/step - loss: 0.3238 - binary_accuracy: 0.8943
Epoch 3/4
769/769 [==============================] - 5s 6ms/step - loss: 0.3230 - binary_accuracy: 0.8943
Epoch 4/4
192/192 [==============================] - 0s 2ms/step - loss: 0.2579 - binary_accuracy: 0.9231
test accuracy: 92.312091588974


# $X_{t-7~t-1}: Y_{t+4}$

In [ ]:
horizon_=4
nhist_=8
feature_name=['Y', 'roff', 'evap']

x_tr,y_tr,x_te,y_te=trte_split(horizon=horizon_,nhist=nhist_,
                               thin=True,
                               trkey=trkey,tekey=tekey,
                               dfs=dfs,
                               feature_name=feature_name)

model=dnnmodel(dim=24,out_dim=1)

model.fit(x_tr,y_tr,batch_size=128,epochs=4)

print('test accuracy:', model.evaluate(x_te,y_te)[1]*100)

Epoch 1/4
769/769 [==============================] - 6s 6ms/step - loss: 0.4430 - binary_accuracy: 0.8327
Epoch 2/4
769/769 [==============================] - 4s 5ms/step - loss: 0.4335 - binary_accuracy: 0.8404
Epoch 3/4
769/769 [==============================] - 4s 5ms/step - loss: 0.4298 - binary_accuracy: 0.8404
Epoch 4/4
192/192 [==============================] - 0s 1ms/step - loss: 0.3415 - binary_accuracy: 0.8871
test accuracy: 88.70915174484253


# $X_{t-15~t-1}: Y_{t+16}$

In [ ]:
horizon_=16
nhist_=26
feature_name=['Y', 'roff', 'evap']

x_tr,y_tr,x_te,y_te=trte_split(horizon=horizon_,nhist=nhist_,
                               thin=True,
                               trkey=trkey,tekey=tekey,
                               dfs=dfs,
                               feature_name=feature_name)

model=dnnmodel(dim=26*3,out_dim=1)

model.fit(x_tr,y_tr,batch_size=128,epochs=4)

print('test accuracy:', model.evaluate(x_te,y_te)[1]*100)

 40%|████      | 145/360 [03:01<07:15,  2.03s/it]

passed county: 45019


100%|██████████| 90/90 [00:06<00:00, 13.57it/s]


Epoch 1/4
685/685 [==============================] - 5s 6ms/step - loss: 0.6324 - binary_accuracy: 0.6596
Epoch 2/4
685/685 [==============================] - 4s 6ms/step - loss: 0.6274 - binary_accuracy: 0.6665
Epoch 3/4
685/685 [==============================] - 4s 6ms/step - loss: 0.6258 - binary_accuracy: 0.6673
Epoch 4/4
107/107 [==============================] - 0s 2ms/step - loss: 0.6115 - binary_accuracy: 0.7436
test accuracy: 74.35672283172607


# TODO: AUC